# Salary Regression 

In [35]:
import tensorflow as tf 
import pickle
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [36]:
# Load the dataset
data = pd.read_csv('dataset/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
data.drop(columns=['RowNumber','CustomerId','Surname'], inplace=True, errors='ignore')
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])
data['Gender'].unique()

array([0, 1])

In [39]:
from sklearn.preprocessing import OneHotEncoder
oe = OneHotEncoder()
geo_encoder = oe.fit_transform(data[['Geography']])

geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [40]:
oe.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [41]:
new_geo = pd.DataFrame(geo_encoder.toarray(), columns=oe.get_feature_names_out())

In [42]:
df = pd.concat([data.drop(columns='Geography'),new_geo], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [43]:
X = df.drop(columns='EstimatedSalary')
y = df['EstimatedSalary']
X_train,X_test, y_train,y_test =  train_test_split(X,y, test_size=0.2, random_state=42)

#scaling the feature
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
sc.feature_names_in_

array(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [44]:
# Saving the encoder

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(le, file)

with open('One_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(oe,file)

with open('scaler_reg.pkl','wb') as file:
    pickle.dump(sc,file)

# ANN Regression implementation

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [46]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape = (X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1)) # Linear activation

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate = 0.01), loss = 'mean_absolute_error', metrics=['mae'])

c:\Users\HARI LOKESH\Desktop\AI projects\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# Set up tensornboard
log_dir = 'reglogs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_calbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [49]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [50]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 100, callbacks=[tensorflow_calbacks, early_stop])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 95477.9922 - mae: 95477.9922 - val_loss: 51125.9219 - val_mae: 51125.9219
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 50045.6875 - mae: 50045.6875 - val_loss: 50541.7578 - val_mae: 50541.7578
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49732.9375 - mae: 49732.9375 - val_loss: 50356.9023 - val_mae: 50356.9023
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49768.8320 - mae: 49768.8320 - val_loss: 50403.4453 - val_mae: 50403.4453
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 50026.4609 - mae: 50026.4609 - val_loss: 50409.8477 - val_mae: 50409.8477
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49678.5234 - mae: 49678.5234 - val_loss: 50400.7422 - val_mae: 50400.7422
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49480.5078 - mae: 49480.5078 - val_loss: 50370.4062 - val_mae: 50370.4062
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - 

In [51]:
test_loss,test_mae = model.evaluate(X_test,y_test)
test_mae

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51345.6406 - mae: 51345.6406


50356.90234375

In [52]:
model.save('reg_model.h5')